## Spotify Playlist Generator

Import/download necessary libraries

In [1]:
pip install -r requirements.txt

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import spotipy
#from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
import sys
import os
import re
from track_info import inject_song_id, search_for_song, initialize_client
from spotify import make_playlist, initialize_auth
from add_song import add_new_songs, update_df


## Generate data frame to use

The data is in a folder named "lyrics/" where there are about 5 songs from the top 500 artists ( you can view the collection process in 'lyric_collector.py'. The name of each file is the artist name, followed by a number, ranging from 1-5 corresponding to their 1st to 5th most popular songs. Each file contains both the song title on the first line, then the song lyrics on the remaining lines.

The goal is to extract this information into a data frame with columns corresponding to artist name, song name, and lyrics.

In [3]:
def extract_info(text, doc_name, details):
    # The format for Happy by Pharrell for example would be :
    # Happy Lyrics
    # {Song Lyrics}
    # Hence, we plit on 'Lyrics'
    try:
        song_name, lyrics = text.split('Lyrics', maxsplit= 1)
        #print(song_name)
    except Exception as e:
        #print(e)
        print(f"{doc_name} has no lyrics or had an error")
        return
    artist, _ = doc_name.rsplit('_', maxsplit = 1)
    details["artist"].append(artist)
    details["song_name"].append(song_name)
    details["lyrics"].append(lyrics)

In [4]:
def read_docs(input_dir):
    details = {
        "artist": [],
        "song_name": [],
        "lyrics": []
    }
    docs = os.listdir(input_dir)
    for doc in docs:
        #print(f"{input_dir}{doc}")
        with open(f"{input_dir}/{doc}") as f:
            doc_text = f.read()
            extract_info(doc_text, doc, details)
    df = pd.DataFrame(details)
    return df


In [5]:
df = read_docs('lyrics/')

In [6]:
df["id"] = df.index
df.head()

,artist,song_name,lyrics,id
0,MGMT,Electric Feel,\nAll along the Western front\nPeople line up ...,0
1,AFI,The Days of the Phoenix,I remember when I was told of story of\nCrushe...,1
2,TVontheRadio,DLZ,\nCongratulations on the mess you made of thin...,2
3,SonicYouth,Schizophrenia,\nI went away to see an old friend of mine\nHi...,3
4,NickCave&TheBadSeeds,O Children,"Pass me that lovely little gun\nMy dear, my da...",4


### Remove Stop words and Tokenize

In [7]:
stop = set()
with open('data/stopwords.txt', 'r') as f:
    for line in f:
        stop.add(line.rstrip())

df['lyrics_nostop'] = df['lyrics'].apply(lambda x: ' '.join([word.lower() for word in x.split() if word not in (stop)]))

In [8]:
df.head()

,artist,song_name,lyrics,id,lyrics_nostop
0,MGMT,Electric Feel,\nAll along the Western front\nPeople line up ...,0,all along western front people line up receive...
1,AFI,The Days of the Phoenix,I remember when I was told of story of\nCrushe...,1,"i remember i told story crushed velvet, candle..."
2,TVontheRadio,DLZ,\nCongratulations on the mess you made of thin...,2,congratulations mess made things i'm trying re...
3,SonicYouth,Schizophrenia,\nI went away to see an old friend of mine\nHi...,3,i went away see old friend mine his sister cam...
4,NickCave&TheBadSeeds,O Children,"Pass me that lovely little gun\nMy dear, my da...",4,"pass lovely little gun my dear, darling one th..."


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
analyze = TfidfVectorizer().build_analyzer()
df['tokens'] = df['lyrics_nostop'].apply(analyze)

Now would be an appropriate time to get rid of some redundant columns

In [11]:
df = df.drop(['lyrics'], axis=1)
df.head()

,artist,song_name,id,lyrics_nostop,tokens
0,MGMT,Electric Feel,0,all along western front people line up receive...,"[all, along, western, front, people, line, up,..."
1,AFI,The Days of the Phoenix,1,"i remember i told story crushed velvet, candle...","[remember, told, story, crushed, velvet, candl..."
2,TVontheRadio,DLZ,2,congratulations mess made things i'm trying re...,"[congratulations, mess, made, things, trying, ..."
3,SonicYouth,Schizophrenia,3,i went away see old friend mine his sister cam...,"[went, away, see, old, friend, mine, his, sist..."
4,NickCave&TheBadSeeds,O Children,4,"pass lovely little gun my dear, darling one th...","[pass, lovely, little, gun, my, dear, darling,..."


## Find similarities between songs and queries

### General cosine similarity based on lyric similarity

In [12]:
query = "I am very happy, give me happy"

q_ser = pd.Series([query])
l_ser = pd.Series(df['lyrics_nostop'])

q_lyrics = q_ser.append(l_ser)

# the linear kernel is the dot product
tf_idf = TfidfVectorizer().fit_transform(q_lyrics)
# the query is the first item in the series, so compare it to the other items
cosine_sim = cosine_similarity(tf_idf[0:1], tf_idf[1:]).flatten()
top_songs_indices = cosine_sim.argsort()[:-7:-1]
print(top_songs_indices)
print(cosine_sim[top_songs_indices])

df.iloc[top_songs_indices]

[1286 2174  449 1174   94 2227]
[0.40734742 0.38548732 0.37506168 0.3115518  0.28000354 0.23671946]


,artist,song_name,id,lyrics_nostop,tokens
1286,DemiLovato,Stone Cold,1286,"stone cold, stone cold you see standing, i'm d...","[stone, cold, stone, cold, you, see, standing,..."
2174,TheKooks,Junk of the Heart (Happy),2174,junk heart junk mind so hard leave alone we ge...,"[junk, heart, junk, mind, so, hard, leave, alo..."
449,SherylCrow,If It Makes You Happy,449,"i've long, long way put poncho, played mosquit...","[ve, long, long, way, put, poncho, played, mos..."
1174,PharrellWilliams,Happy,1174,it might seem crazy i'm 'bout say sunshine she...,"[it, might, seem, crazy, bout, say, sunshine, ..."
94,TheWombats,Let’s Dance to Joy Division,94,i'm back liverpool everything seems same but i...,"[back, liverpool, everything, seems, same, but..."
2227,CatStevens,Tea for the Tillerman,2227,bring tea tillerman steak sun wine woman made ...,"[bring, tea, tillerman, steak, sun, wine, woma..."


If you look at the lyrics, key words like "happy" and "give" show up a lot, but the query is only stop words and those two words, and there are no stop words in the lyrics! So we need to look at some other methods, first lets make this a function.

In [13]:
def get_songs_from_lyrics(query, df, amnt):
    q_ser = pd.Series([query])
    l_ser = pd.Series(df['lyrics_nostop'])

    q_lyrics = q_ser.append(l_ser)

    # the linear kernel is the dot product
    tf_idf = TfidfVectorizer().fit_transform(q_lyrics)
    # the query is the first item in the series, so compare it to the other items
    cosine_sim = cosine_similarity(tf_idf[0:1], tf_idf[1:]).flatten()
    top_songs_indices = cosine_sim.argsort()[:-amnt:-1]

    df.iloc[top_songs_indices]

My idea is to incorporate some data from spotify, as they have some nice metrics such as 'danceability', 'energy', 'loudness' etc. If we create a vector for each song, we can use a similar method to the one above, where we measure the 'closeness' of each vector. The query in this case would have to be a song, so we can extract the same metrics and measure closeness.

## Connecting to Spotify

In [14]:
cid = '1d668be1930e487eaacd284df4fa7601'
secret = '08ac712c04ba4bffaeb232efe98a7a54'

sp = initialize_client(cid,secret)

In [15]:
# test functionality
song = search_for_song(sp, "Pharrell Williams", "Happy")
print(song)

60nZcImufyMA1MKQY3dcCH


In [16]:
# pretty sure this isn't a song
song = search_for_song(sp, "Beep Boop", "Bop")
print(song)

nan


We can now use this id to extract certain information, but we want this information for all of the songs in our dataset.

In [17]:
df["artist_tokens"] = df["artist"].apply(lambda x: re.findall('([A-Z.!,$\'][a-z]*)', x))

In [18]:
df.head()

,artist,song_name,id,lyrics_nostop,tokens,artist_tokens
0,MGMT,Electric Feel,0,all along western front people line up receive...,"[all, along, western, front, people, line, up,...","[M, G, M, T]"
1,AFI,The Days of the Phoenix,1,"i remember i told story crushed velvet, candle...","[remember, told, story, crushed, velvet, candl...","[A, F, I]"
2,TVontheRadio,DLZ,2,congratulations mess made things i'm trying re...,"[congratulations, mess, made, things, trying, ...","[T, Vonthe, Radio]"
3,SonicYouth,Schizophrenia,3,i went away see old friend mine his sister cam...,"[went, away, see, old, friend, mine, his, sist...","[Sonic, Youth]"
4,NickCave&TheBadSeeds,O Children,4,"pass lovely little gun my dear, darling one th...","[pass, lovely, little, gun, my, dear, darling,...","[Nick, Cave, The, Bad, Seeds]"


### Cleaning and preprocessing

The naming scheme produces some problems, SonicYouth wont turn up any results, but Sonic Youth will, so we need to split somehow, but doing it by capital letters isn't good because bands like MGMT with be M G M T which another problem. So we can get the original name by merging on the same transformation.

In [19]:
artists = pd.read_csv("data/top_artists.csv")
artists = artists[0:600]
artists.head()

,Unnamed: 0,artist_mb,listeners_lastfm
0,0,Coldplay,5381567.0
1,1,Radiohead,4732528.0
2,2,Red Hot Chili Peppers,4620835.0
3,3,Rihanna,4558193.0
4,4,Eminem,4517997.0


In [20]:
# peform same transformation that filenaming used to merge onto other data set
artists["artist"] = artists["artist_mb"].apply(lambda x: x.replace(' ',''))

In [21]:
artists.head()

,Unnamed: 0,artist_mb,listeners_lastfm,artist
0,0,Coldplay,5381567.0,Coldplay
1,1,Radiohead,4732528.0,Radiohead
2,2,Red Hot Chili Peppers,4620835.0,RedHotChiliPeppers
3,3,Rihanna,4558193.0,Rihanna
4,4,Eminem,4517997.0,Eminem


Notice now we have the original artist name, and the transformed version, now we can merge

In [22]:
#do an inner merge so we get the intersection, and there are no inconsistoncies
info = pd.merge(artists, df, on = "artist")

In [23]:
print(info.shape)

(2410, 9)


In [24]:
info.head(20)

,Unnamed: 0,artist_mb,listeners_lastfm,artist,song_name,id,lyrics_nostop,tokens,artist_tokens
0,0,Coldplay,5381567.0,Coldplay,Viva la Vida,399,i used rule world seas would rise i gave word ...,"[used, rule, world, seas, would, rise, gave, w...",[Coldplay]
1,0,Coldplay,5381567.0,Coldplay,Hymn for the Weekend,787,"and said drink me, drink (oh-ah-oh-ah) that we...","[and, said, drink, me, drink, oh, ah, oh, ah, ...",[Coldplay]
2,0,Coldplay,5381567.0,Coldplay,The Scientist,1075,"come up meet you, tell i'm sorry you don't kno...","[come, up, meet, you, tell, sorry, you, don, k...",[Coldplay]
3,0,Coldplay,5381567.0,Coldplay,Yellow,2120,"look stars, look shine and everything do yeah,...","[look, stars, look, shine, and, everything, do...",[Coldplay]
4,0,Coldplay,5381567.0,Coldplay,Fix You,2169,"when try best, don't succeed when get want, no...","[when, try, best, don, succeed, when, get, wan...",[Coldplay]
5,1,Radiohead,4732528.0,Radiohead,Creep,558,when were before couldn't look eye you're just...,"[when, were, before, couldn, look, eye, you, r...",[Radiohead]
6,1,Radiohead,4732528.0,Radiohead,Exit Music (For A Film),630,"wake sleep the drying tears today we escape, w...","[wake, sleep, the, drying, tears, today, we, e...",[Radiohead]
7,1,Radiohead,4732528.0,Radiohead,Karma Police,1221,"karma police, arrest man he talks maths, buzze...","[karma, police, arrest, man, he, talks, maths,...",[Radiohead]
8,1,Radiohead,4732528.0,Radiohead,No Surprises,1965,a heart that's full up like landfill a job slo...,"[heart, that, full, up, like, landfill, job, s...",[Radiohead]
9,1,Radiohead,4732528.0,Radiohead,Paranoid Android,2321,"please, could stop noise? i'm trying get rest ...","[please, could, stop, noise, trying, get, rest...",[Radiohead]


In [25]:
drop = ['artist', 'artist_tokens', 'listeners_lastfm']
info = info.drop(drop, axis = 1)
info.head()

,Unnamed: 0,artist_mb,song_name,id,lyrics_nostop,tokens
0,0,Coldplay,Viva la Vida,399,i used rule world seas would rise i gave word ...,"[used, rule, world, seas, would, rise, gave, w..."
1,0,Coldplay,Hymn for the Weekend,787,"and said drink me, drink (oh-ah-oh-ah) that we...","[and, said, drink, me, drink, oh, ah, oh, ah, ..."
2,0,Coldplay,The Scientist,1075,"come up meet you, tell i'm sorry you don't kno...","[come, up, meet, you, tell, sorry, you, don, k..."
3,0,Coldplay,Yellow,2120,"look stars, look shine and everything do yeah,...","[look, stars, look, shine, and, everything, do..."
4,0,Coldplay,Fix You,2169,"when try best, don't succeed when get want, no...","[when, try, best, don, succeed, when, get, wan..."


In [26]:
# Avoid running this because this is written to a csv
# all work is done in track_info.py
# Uncomment if running for the first time
# inject_song_id(sp, info)

See track_info.py to view the process of injecting these ids into the dataframe. The process took about 3 minutes, so I figured it was easier to store in a csv for easy access.

In [27]:
info = pd.read_csv("data/data_with_spotify_ids.csv")

In [28]:
info["sp_id"].isna().sum()

0

That did the trick, we will simply remove these last 79 songs because there are likely errors that would waste a lot of time to fix.

In [29]:
info.shape

(2331, 8)

Now we have 2,331 songs to build our playlists on, and there is always a way to add artists I/you like if I needed to later.

### Add some song metadata

I already processed and extracted all of the data into a csv, see track_info.py to see how that was done

Now we have some features that will predict song similarity!

In [30]:
track_info = pd.read_csv("data/audio_and_lyric_data.csv")

In [31]:
track_info = track_info.drop("Unnamed: 0",axis=1)
track_info["song_name"] = track_info["song_name"].apply(lambda x: x.lower().strip())
track_info["artist_mb"] = track_info["artist_mb"].apply(lambda x: x.lower().strip())
track_info.drop_duplicates(subset = ["song_name"], keep = 'first')
track_info.head()

,artist_mb,song_name,id,lyrics_nostop,tokens,sp_id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,coldplay,viva la vida,399,i used rule world seas would rise i gave word ...,"['used', 'rule', 'world', 'seas', 'would', 'ri...",1mea3bSkSGXuIRvnydlB5b,0.486,0.617,5.0,-7.115,0.0287,0.09540,0.000003,0.109,0.417,138.015
1,coldplay,hymn for the weekend,787,"and said drink me, drink (oh-ah-oh-ah) that we...","['and', 'said', 'drink', 'me', 'drink', 'oh', ...",3RiPr603aXAoi4GHyXx0uy,0.491,0.693,0.0,-6.487,0.0377,0.21100,0.000007,0.325,0.412,90.027
2,coldplay,the scientist,1075,"come up meet you, tell i'm sorry you don't kno...","['come', 'up', 'meet', 'you', 'tell', 'sorry',...",75JFxkI2RXiU7L9VXzMkle,0.557,0.442,5.0,-7.224,0.0243,0.73100,0.000015,0.110,0.213,146.277
3,coldplay,yellow,2120,"look stars, look shine and everything do yeah,...","['look', 'stars', 'look', 'shine', 'and', 'eve...",3AJwUDP919kvQ9QcozQPxg,0.429,0.661,11.0,-7.227,0.0281,0.00239,0.000121,0.234,0.285,173.372
4,coldplay,fix you,2169,"when try best, don't succeed when get want, no...","['when', 'try', 'best', 'don', 'succeed', 'whe...",7LVHVU3tWfcxj5aiPFEW4Q,0.209,0.417,3.0,-8.740,0.0338,0.16400,0.001960,0.113,0.124,138.178


In [32]:
track_info.isna().sum()

artist_mb           0
song_name           0
id                  0
lyrics_nostop       0
tokens              0
sp_id               0
danceability        0
energy              0
key                 0
loudness            0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
dtype: int64

Now we have a lot of information on the song to work with!

### Jump back to lyrics for a bit

We can modify a function from above to actually generate a list of spotify song ids, and then we can easily make an actual playlist!

In [33]:
def get_songs_from_lyrics(query, df, amnt):
    query = query.lower()
    q_ser = pd.Series([query])
    l_ser = pd.Series(df['lyrics_nostop'])

    q_lyrics = q_ser.append(l_ser)

    # the linear kernel is the dot product
    tf_idf = TfidfVectorizer().fit_transform(q_lyrics)
    # the query is the first item in the series, so compare it to the other items
    cosine_sim = cosine_similarity(tf_idf[0:1], tf_idf[1:]).flatten()
    top_songs_indices = cosine_sim.argsort()[:-amnt:-1]

    return list(df.iloc[top_songs_indices]["sp_id"])

In [34]:
songs = get_songs_from_lyrics("Happy Fun Time YAY", track_info, 15)
print(songs)

['2jZgzSxNSg1hTCq0ewWHGJ', '5XrfFo0JFOnWD9ZMNXGkQh', '4y1LsJpmMti1PfRQV9AWWe', '3by8IfnW9dZ2t4pZw1WVxz', '60nZcImufyMA1MKQY3dcCH', '3ZpQiJ78LKINrW9SQTgbXd', '5X8kkUaUlAyAUr9TYqDFTH', '60Ih7J2Q1o9shvMC2OAALu', '6kex4EBAj0WHXDKZMEJaaF', '6aPFJ2b6iApBxXHNFzzYAB', '26uDUal1TkjwwOaQQt1Uwy', '7o9uu2GDtVDr9nsR7ZRN73', '1IsGA5ceSC4a5nxgAEYnQd', '7jKsYUxH1F0WWV7nPSReRb']


In [35]:
spa = initialize_auth(cid, secret)

In [36]:
u_id = '6046owg3hsttaxfq2ot19rrea'

This is giving me tooo many problems at the moment so we will come back to it...maybe

In [37]:

# name = "Happy Fun Times"
# playlist, playlist_url = make_playlist(spa, u_id, name, songs)

## Find songs based on audio features

In [38]:
track_info.head()

,artist_mb,song_name,id,lyrics_nostop,tokens,sp_id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,coldplay,viva la vida,399,i used rule world seas would rise i gave word ...,"['used', 'rule', 'world', 'seas', 'would', 'ri...",1mea3bSkSGXuIRvnydlB5b,0.486,0.617,5.0,-7.115,0.0287,0.09540,0.000003,0.109,0.417,138.015
1,coldplay,hymn for the weekend,787,"and said drink me, drink (oh-ah-oh-ah) that we...","['and', 'said', 'drink', 'me', 'drink', 'oh', ...",3RiPr603aXAoi4GHyXx0uy,0.491,0.693,0.0,-6.487,0.0377,0.21100,0.000007,0.325,0.412,90.027
2,coldplay,the scientist,1075,"come up meet you, tell i'm sorry you don't kno...","['come', 'up', 'meet', 'you', 'tell', 'sorry',...",75JFxkI2RXiU7L9VXzMkle,0.557,0.442,5.0,-7.224,0.0243,0.73100,0.000015,0.110,0.213,146.277
3,coldplay,yellow,2120,"look stars, look shine and everything do yeah,...","['look', 'stars', 'look', 'shine', 'and', 'eve...",3AJwUDP919kvQ9QcozQPxg,0.429,0.661,11.0,-7.227,0.0281,0.00239,0.000121,0.234,0.285,173.372
4,coldplay,fix you,2169,"when try best, don't succeed when get want, no...","['when', 'try', 'best', 'don', 'succeed', 'whe...",7LVHVU3tWfcxj5aiPFEW4Q,0.209,0.417,3.0,-8.740,0.0338,0.16400,0.001960,0.113,0.124,138.178


In [39]:
track_info = update_df()

In [40]:
track_info

,Unnamed: 0,artist_mb,song_name,id,lyrics_nostop,tokens,sp_id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0,coldplay,viva la vida,399,i used rule world seas would rise i gave word ...,"['used', 'rule', 'world', 'seas', 'would', 'ri...",1mea3bSkSGXuIRvnydlB5b,0.486,0.617,5.0,-7.115,0.0287,0.09540,0.000003,0.1090,0.417,138.015
1,1,coldplay,hymn for the weekend,787,"and said drink me, drink (oh-ah-oh-ah) that we...","['and', 'said', 'drink', 'me', 'drink', 'oh', ...",3RiPr603aXAoi4GHyXx0uy,0.491,0.693,0.0,-6.487,0.0377,0.21100,0.000007,0.3250,0.412,90.027
2,2,coldplay,the scientist,1075,"come up meet you, tell i'm sorry you don't kno...","['come', 'up', 'meet', 'you', 'tell', 'sorry',...",75JFxkI2RXiU7L9VXzMkle,0.557,0.442,5.0,-7.224,0.0243,0.73100,0.000015,0.1100,0.213,146.277
3,3,coldplay,yellow,2120,"look stars, look shine and everything do yeah,...","['look', 'stars', 'look', 'shine', 'and', 'eve...",3AJwUDP919kvQ9QcozQPxg,0.429,0.661,11.0,-7.227,0.0281,0.00239,0.000121,0.2340,0.285,173.372
4,4,coldplay,fix you,2169,"when try best, don't succeed when get want, no...","['when', 'try', 'best', 'don', 'succeed', 'whe...",7LVHVU3tWfcxj5aiPFEW4Q,0.209,0.417,3.0,-8.740,0.0338,0.16400,0.001960,0.1130,0.124,138.178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2344,2344,king gizzard & the lizard wizard,work this time,1,i knock hello but i see you've got hypothermia...,"['knock', 'hello', 'but', 'see', 'you', 've', ...",4VtNDouDk6gLgopBhp9di6,0.529,0.491,1.0,-6.249,0.1930,0.15400,0.100000,0.1360,0.540,113.996
2345,2345,king gizzard & the lizard wizard,rattlesnake,2,"rattlesnake, rattlesnake rattlesnake, rattles ...","['rattlesnake', 'rattlesnake', 'rattlesnake', ...",4HOnIpqP8txGle3UvfjStU,0.582,0.962,11.0,-5.924,0.0388,0.03090,0.841000,0.3460,0.814,170.555
2346,2346,king gizzard & the lizard wizard,robot stop,3,nonagon infinity opens door nonagon infinity o...,"['nonagon', 'infinity', 'opens', 'door', 'nona...",4bbZcD92nLvQnbrblZTS1K,0.465,0.988,9.0,-5.301,0.1100,0.00565,0.198000,0.8600,0.315,97.920
2347,2347,king gizzard & the lizard wizard,crumbling castle,5,"meet glassy-eyed, blind, divine hold down fort...","['meet', 'glassy', 'eyed', 'blind', 'divine', ...",4T1168WOWj7Ch7DbbNEFp7,0.359,0.727,6.0,-7.252,0.0433,0.14700,0.029900,0.0711,0.578,97.659


In [41]:
from scipy.spatial import distance

In [42]:
def get_query_song_info(daf, artist, song):
    artist_songs = daf[daf["artist_mb"] == artist.lower().strip()]
    print(song.lower().strip())
    song_info = daf[daf['song_name'] == song.lower().strip()]
    print(song_info.head())
    return pd.merge(artist_songs, song_info)

In [43]:
get_query_song_info(track_info, 'King Gizzard & The Lizard Wizard', "Robot Stop")

robot stop
      Unnamed: 0                         artist_mb   song_name  id  \
2346        2346  king gizzard & the lizard wizard  robot stop   3   

                                          lyrics_nostop  \
2346  nonagon infinity opens door nonagon infinity o...   

                                                 tokens  \
2346  ['nonagon', 'infinity', 'opens', 'door', 'nona...   

                       sp_id  danceability  energy  key  loudness  \
2346  4bbZcD92nLvQnbrblZTS1K         0.465   0.988  9.0    -5.301   

      speechiness  acousticness  instrumentalness  liveness  valence  tempo  
2346         0.11       0.00565             0.198      0.86    0.315  97.92  


,Unnamed: 0,artist_mb,song_name,id,lyrics_nostop,tokens,sp_id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2346,king gizzard & the lizard wizard,robot stop,3,nonagon infinity opens door nonagon infinity o...,"['nonagon', 'infinity', 'opens', 'door', 'nona...",4bbZcD92nLvQnbrblZTS1K,0.465,0.988,9.0,-5.301,0.11,0.00565,0.198,0.86,0.315,97.92


In [44]:
artist = "Coldplay"
song = "Yellow"

good_song = get_query_song_info(track_info, artist, song)
good_song

yellow
   Unnamed: 0 artist_mb song_name    id  \
3           3  coldplay    yellow  2120   

                                       lyrics_nostop  \
3  look stars, look shine and everything do yeah,...   

                                              tokens                   sp_id  \
3  ['look', 'stars', 'look', 'shine', 'and', 'eve...  3AJwUDP919kvQ9QcozQPxg   

   danceability  energy   key  loudness  speechiness  acousticness  \
3         0.429   0.661  11.0    -7.227       0.0281       0.00239   

   instrumentalness  liveness  valence    tempo  
3          0.000121     0.234    0.285  173.372  


,Unnamed: 0,artist_mb,song_name,id,lyrics_nostop,tokens,sp_id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3,coldplay,yellow,2120,"look stars, look shine and everything do yeah,...","['look', 'stars', 'look', 'shine', 'and', 'eve...",3AJwUDP919kvQ9QcozQPxg,0.429,0.661,11.0,-7.227,0.0281,0.00239,0.000121,0.234,0.285,173.372


In [45]:
artist = "CNJEFWK"
song = "fjke"

bad_song = get_query_song_info(track_info, artist, song)
bad_song

fjke
Empty DataFrame
Columns: [Unnamed: 0, artist_mb, song_name, id, lyrics_nostop, tokens, sp_id, danceability, energy, key, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo]
Index: []


,Unnamed: 0,artist_mb,song_name,id,lyrics_nostop,tokens,sp_id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo


Now we have our queries information, we need to compare it to all the other songs, to start we will use Euclidean distance

In [46]:
def song_sim(song1, song2):
    return distance.braycurtis(song1, song2)

In [47]:
track_info.columns

Index(['Unnamed: 0', 'artist_mb', 'song_name', 'id', 'lyrics_nostop', 'tokens',
       'sp_id', 'danceability', 'energy', 'key', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype='object')

In [48]:
compare = ['danceability', 'energy', 'key', 'loudness', 'speechiness','acousticness', 
            'instrumentalness', 'liveness', 'valence', 'tempo']

In [49]:
comp_query = good_song[compare]
comp_query = comp_query.iloc[0]
comp_query = comp_query.to_list()


In [50]:
comp_df = track_info[compare]
comp_df = comp_df.apply(pd.to_numeric)


In [51]:
# distance = comp_df.apply(lambda x : 
#             song_sim(list(x), comp_query.to_list()), axis = 1)

In [52]:

track_info["distance_from_query"] = comp_df.apply(lambda x : 
            song_sim(x.to_list(), 
            comp_query), axis = 1)

In [53]:
track_info.head()

,Unnamed: 0,artist_mb,song_name,id,lyrics_nostop,tokens,sp_id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,distance_from_query
0,0,coldplay,viva la vida,399,i used rule world seas would rise i gave word ...,"['used', 'rule', 'world', 'seas', 'would', 'ri...",1mea3bSkSGXuIRvnydlB5b,0.486,0.617,5.0,-7.115,0.0287,0.09540,0.000003,0.109,0.417,138.015,0.121467
1,1,coldplay,hymn for the weekend,787,"and said drink me, drink (oh-ah-oh-ah) that we...","['and', 'said', 'drink', 'me', 'drink', 'oh', ...",3RiPr603aXAoi4GHyXx0uy,0.491,0.693,0.0,-6.487,0.0377,0.21100,0.000007,0.325,0.412,90.027,0.327537
2,2,coldplay,the scientist,1075,"come up meet you, tell i'm sorry you don't kno...","['come', 'up', 'meet', 'you', 'tell', 'sorry',...",75JFxkI2RXiU7L9VXzMkle,0.557,0.442,5.0,-7.224,0.0243,0.73100,0.000015,0.110,0.213,146.277,0.097151
3,3,coldplay,yellow,2120,"look stars, look shine and everything do yeah,...","['look', 'stars', 'look', 'shine', 'and', 'eve...",3AJwUDP919kvQ9QcozQPxg,0.429,0.661,11.0,-7.227,0.0281,0.00239,0.000121,0.234,0.285,173.372,0.000000
4,4,coldplay,fix you,2169,"when try best, don't succeed when get want, no...","['when', 'try', 'best', 'don', 'succeed', 'whe...",7LVHVU3tWfcxj5aiPFEW4Q,0.209,0.417,3.0,-8.740,0.0338,0.16400,0.001960,0.113,0.124,138.178,0.132538


In [54]:
track_info = track_info.sort_values(by = ['distance_from_query'])
track_info.head(20)

,Unnamed: 0,artist_mb,song_name,id,lyrics_nostop,tokens,sp_id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,distance_from_query
3,3,coldplay,yellow,2120,"look stars, look shine and everything do yeah,...","['look', 'stars', 'look', 'shine', 'and', 'eve...",3AJwUDP919kvQ9QcozQPxg,0.429,0.661,11.0,-7.227,0.0281,0.002390,0.000121,0.2340,0.285,173.372,0.000000
1840,1840,judas priest,beyond the realms of death,1817,"he had enough, couldn't take anymore he found ...","['he', 'had', 'enough', 'couldn', 'take', 'any...",3Rtv6zRNKpeSygXoaF9kCm,0.287,0.716,11.0,-5.653,0.0311,0.038100,0.000032,0.1640,0.544,172.123,0.008827
690,690,deep purple,highway star,541,nobody gonna take car i'm gonna race ground no...,"['nobody', 'gonna', 'take', 'car', 'gonna', 'r...",4gVTozEmzwAUXpwj3jEetX,0.298,0.895,9.0,-7.647,0.0547,0.028300,0.005740,0.3050,0.544,173.737,0.009172
2020,2020,kelis,millionaire,2024,"i said, city so got witty, witty she said, cou...","['said', 'city', 'so', 'got', 'witty', 'witty'...",1mVTQSGoFchL3VVrIa0MeH,0.686,0.855,11.0,-6.600,0.1160,0.025200,0.000000,0.0370,0.763,176.002,0.011543
509,509,evanescence,going under,770,now i tell i've done fifty thousand tears i've...,"['now', 'tell', 've', 'done', 'fifty', 'thousa...",3UygY7qW2cvG9Llkay6i1i,0.370,0.858,11.0,-4.885,0.0545,0.008150,0.000021,0.2290,0.464,175.077,0.011702
206,206,the cure,pictures of you,828,i've looking so long these pictures that i alm...,"['ve', 'looking', 'so', 'long', 'these', 'pict...",1tuwC1Ob5vnYZhvg3zyzJg,0.484,0.776,9.0,-6.480,0.0312,0.003710,0.109000,0.1310,0.495,171.935,0.012492
256,256,nickelback,how you remind me,573,never made wise man i couldn't cut poor man st...,"['never', 'made', 'wise', 'man', 'couldn', 'cu...",0gmbgwZ8iqyMPmXefof8Yf,0.446,0.764,10.0,-5.042,0.0330,0.001350,0.000000,0.0990,0.543,172.094,0.013033
473,473,keane,nothing in my way,76,a turning tide lovers great divide why do laug...,"['turning', 'tide', 'lovers', 'great', 'divide...",7falOwHJ9MjskdcNehIh79,0.399,0.854,10.0,-4.970,0.0294,0.001070,0.000004,0.0893,0.872,172.478,0.013340
1333,1333,duran duran,come undone,1558,"mine, immaculate dream made breath skin, i've ...","['mine', 'immaculate', 'dream', 'made', 'breat...",0yfNXxlyXdmP0ue1iJijx1,0.542,0.793,8.0,-8.203,0.0530,0.042300,0.000004,0.0814,0.708,173.931,0.014058
1419,1419,stone temple pilots,interstate love song,1678,waiting sunday afternoon for i've read between...,"['waiting', 'sunday', 'afternoon', 'for', 've'...",6qLEOZvf5gI7kWE63JE7p3,0.215,0.927,9.0,-7.327,0.0452,0.000273,0.006610,0.1260,0.490,170.814,0.014330


So after picking a random song, "Yellow" by Coldplay, we now have the top 20 songs that match the audio features of that song. After listening to "Yellow", "Beyond the Realms of Death", "Highway Star", and "Millionaire", I can confirm that they are similar!

In [60]:
def songs_with_similar_audio_features(track_info, amnt):
    compare = ['danceability', 'energy', 'key', 'loudness', 'speechiness','acousticness', 
            'instrumentalness', 'liveness', 'valence', 'tempo']
    artist = input("Input an artists name: ")
    artist = artist.lower().strip()
    if not track_info['artist_mb'].str.contains(artist).any():
        print("That artist isn't in the database, let's try adding it!")
        add_new_songs()
    print("We have some songs from them!")
    songs = track_info[track_info["artist_mb"] == artist]
    for song in list(songs["song_name"]):
        print(song)
    song = input("Input one of their songs: ")

    result = get_query_song_info(track_info, artist, song)
    if result.empty:
        print("Looks like there was a problem, try again")
        return
    comp_query = result[compare]
    comp_query = comp_query.iloc[0]
    comp_query = comp_query.to_list()

    comp_df = track_info[compare]
    comp_df = comp_df.apply(pd.to_numeric)

    
    track_info["distance_from_query"] = comp_df.apply(lambda x : 
            song_sim(x.to_list(), comp_query), axis = 1)
    
    top_tracks = track_info.sort_values(by = ['distance_from_query'])

    return top_tracks[0:amnt+1]


In [56]:
track_info['artist_mb'].str.contains('King Gizzard & the Lizard Wizard').any()

False

In [57]:
def audio_similarity_spotify_ids(top_songs):
    return list(top_songs["sp_id"])

In [58]:
get_query_song_info(track_info, 'Tame Impala', 'The Less I Know the Better')

the less i know the better
      Unnamed: 0    artist_mb                   song_name    id  \
2343        2343  tame impala  the less i know the better  2429   

                                          lyrics_nostop  \
2343  someone said left together i ran out door get ...   

                                                 tokens  \
2343  ['someone', 'said', 'left', 'together', 'ran',...   

                       sp_id  danceability  energy  key  loudness  \
2343  6K4t31amVTZDgR3sKmwUJJ          0.64    0.74  4.0    -4.083   

      speechiness  acousticness  instrumentalness  liveness  valence    tempo  \
2343       0.0284        0.0115           0.00678     0.167    0.785  116.879   

      distance_from_query  
2343             0.210588  


,Unnamed: 0,artist_mb,song_name,id,lyrics_nostop,tokens,sp_id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,distance_from_query
0,2343,tame impala,the less i know the better,2429,someone said left together i ran out door get ...,"['someone', 'said', 'left', 'together', 'ran',...",6K4t31amVTZDgR3sKmwUJJ,0.64,0.74,4.0,-4.083,0.0284,0.0115,0.00678,0.167,0.785,116.879,0.210588


In [61]:
top_songs = songs_with_similar_audio_features(track_info, 20)

We have some songs from them!
rattlesnake
work this time
robot stop
gamma knife
crumbling castle
robot stop
      Unnamed: 0                         artist_mb   song_name  id  \
2346        2346  king gizzard & the lizard wizard  robot stop   3   

                                          lyrics_nostop  \
2346  nonagon infinity opens door nonagon infinity o...   

                                                 tokens  \
2346  ['nonagon', 'infinity', 'opens', 'door', 'nona...   

                       sp_id  danceability  energy  key  loudness  \
2346  4bbZcD92nLvQnbrblZTS1K         0.465   0.988  9.0    -5.301   

      speechiness  acousticness  instrumentalness  liveness  valence  tempo  \
2346         0.11       0.00565             0.198      0.86    0.315  97.92   

      distance_from_query  
2346             0.261607  


In [62]:
top_songs

,Unnamed: 0,artist_mb,song_name,id,lyrics_nostop,tokens,sp_id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,distance_from_query
2346,2346,king gizzard & the lizard wizard,robot stop,3,nonagon infinity opens door nonagon infinity o...,"['nonagon', 'infinity', 'opens', 'door', 'nona...",4bbZcD92nLvQnbrblZTS1K,0.465,0.988,9.0,-5.301,0.1100,0.00565,0.198000,0.8600,0.315,97.920,0.000000
2118,2118,faith no more,evidence,967,if want open hole just put head down go and st...,"['if', 'want', 'open', 'hole', 'just', 'put', ...",5Z7o8zLaEBgpyOTZPp0jj9,0.678,0.801,9.0,-6.367,0.0310,0.04600,0.007180,0.0765,0.627,98.008,0.012823
60,60,lady gaga,do what u want,920,"yeah (oh) turn mic up (yeah, yeah) eh eh eh eh...","['yeah', 'oh', 'turn', 'mic', 'up', 'yeah', 'y...",5XKXMWPACPq51OiqzxenZo,0.778,0.801,9.0,-4.650,0.0860,0.34500,0.000001,0.0881,0.534,97.470,0.013775
1042,1042,hoobastank,running away,1279,i don't want give up and leave own life collec...,"['don', 'want', 'give', 'up', 'and', 'leave', ...",6xdSzlVVKW3fQidPxBImTG,0.514,0.819,9.0,-5.475,0.0418,0.00101,0.000002,0.1950,0.488,100.400,0.017152
1764,1764,trey songz,na na,2256,"put hands air, air trigga put your, put hands ...","['put', 'hands', 'air', 'air', 'trigga', 'put'...",5maiP9UMnTSgoxPX8X3bdz,0.670,0.476,9.0,-6.253,0.0406,0.32800,0.000000,0.1040,0.235,96.975,0.017622
1676,1676,colbie caillat,brighter than the sun,992,"stop corner swear hit like vision i, i, i wasn...","['stop', 'corner', 'swear', 'hit', 'like', 'vi...",5YTtTbqxUzJHD3CpbNHerP,0.629,0.793,9.0,-5.942,0.0346,0.00952,0.000001,0.2180,0.588,95.981,0.018091
781,781,the game,hate it or love it,1111,"yeah, let's take 'em back uh-huh coming up i c...","['yeah', 'let', 'take', 'em', 'back', 'uh', 'h...",2wGSgTmgSF3xjRrHkTc25R,0.802,0.785,9.0,-4.781,0.2070,0.14000,0.000000,0.1230,0.435,99.998,0.019117
1800,1800,p.o.d.,youth of the nation,153,last day rest life i wish i would've known 'ca...,"['last', 'day', 'rest', 'life', 'wish', 'would...",0UrWr7Jnu1heq1o99ZwUd0,0.563,0.860,8.0,-7.533,0.0621,0.00834,0.010600,0.3900,0.517,97.867,0.019141
662,662,gwen stefani,used to love you,2329,"never thought would happen gotta let sink in, ...","['never', 'thought', 'would', 'happen', 'gotta...",7dhB1xznh2bbCWrFjCSH5x,0.623,0.654,8.0,-5.408,0.0674,0.01830,0.000002,0.3980,0.193,96.017,0.019154
2177,2177,brand new,cant get it out,494,as we glide over whatever we know over forever...,"['as', 'we', 'glide', 'over', 'whatever', 'we'...",03gwBLH630cKYF1gr6hMGF,0.557,0.716,8.0,-2.782,0.0302,0.00333,0.000225,0.6610,0.261,97.977,0.019778


In [ ]:
audio_similarity_spotify_ids(top_songs)

['6K4t31amVTZDgR3sKmwUJJ',
 '3LrKEPhAAkP8tkVqgh3dp0',
 '1XTGyfJeMiZXrZ1W3NolcB',
 '4Rm0V366wsNJovOqCYOu5O',
 '1Jhbr1Ttq14xkPqkhTER33',
 '2WIxwn8LZRI7knO1VbwdaE',
 '1le5KVGTF1xWf2aUj7ruLy',
 '5R8dQOPq8haW94K7mgERlO',
 '25CMmGsl22APKhfuj4Tp7j',
 '60Ih7J2Q1o9shvMC2OAALu',
 '6AdLyvisRKHLAwARaZMGr6',
 '2CEgGE6aESpnmtfiZwYlbV',
 '0cahtHEywK64NSWqm2SDTi',
 '3KAS4vmuvRGP2BUQcxmu5i',
 '5ghIJDpPoe3CfHMGu71E6T',
 '215JYyyUnrJ98NK3KEwu6d',
 '0Qv7xi6uPSqH2k82tOkGSt',
 '1D3YGmST01Ba0WRiT7THPG',
 '0ItD5cgfl9jvzVTOlehFxd',
 '4v2rkl1mC3zVAz0nXMx9r4',
 '1kJtfldK9F7XmsSiSNlbth']

## Now we have all the features to recommend playlists

In [ ]:
add_new_songs()
track_info = update_df()

NameError: name 'add_new_songs' is not defined

In [ ]:
def get_songs_from_lyrics(df, amnt):
    query = input("Input your mood, favorite words, or favorite lyrics!")
    query = query.lower()
    q_ser = pd.Series([query])
    l_ser = pd.Series(df['lyrics_nostop'])

    q_lyrics = q_ser.append(l_ser)

    # the linear kernel is the dot product
    tf_idf = TfidfVectorizer().fit_transform(q_lyrics)
    # the query is the first item in the series, so compare it to the other items
    cosine_sim = cosine_similarity(tf_idf[0:1], tf_idf[1:]).flatten()
    top_songs_indices = cosine_sim.argsort()[:-amnt:-1]

    return df.iloc[top_songs_indices]

In [ ]:
def recommend(track_info):
    type_sim = input("Would you like to find similar songs based on a query and lyrics (L)\n \
    or would you like to enter a song and find similar songs that way (S)?")
    if type_sim == 'S':
        top_songs = songs_with_similar_audio_features(track_info, 20)
    elif type_sim == 'L':
        top_songs = get_songs_from_lyrics(track_info, 20)
    else:
        print("Not a valid option!")
        return
    print("Listen to these: ")
    top_songs.apply(lambda x : print(f"{x$song_name} by {x$artist_mb}"]))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=020269be-b327-479a-994d-3e58f34cfdc4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>